In [1]:
# RESETEAR TODO PARA PARTIR LIMPIO EN COLAB

import os, shutil

# 1) Ir a /content
%cd /content

# 2) Borrar cualquier clone previo duplicado
if os.path.exists("proyecto_aplicado_preservantes"):
    shutil.rmtree("proyecto_aplicado_preservantes")
    print("🗑️ Carpeta borrada: proyecto_aplicado_preservantes")

# 3) Clonar de nuevo desde tu GitHub
!git clone https://github.com/nalpata/proyecto_aplicado_preservantes.git

# 4) Entrar a la carpeta correcta
%cd proyecto_aplicado_preservantes

print("\n🎉 Listo. Ahora estamos en el repo correcto sin duplicados.")
!ls


/content
Cloning into 'proyecto_aplicado_preservantes'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 112 (delta 32), reused 77 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (112/112), 19.32 MiB | 36.03 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/proyecto_aplicado_preservantes

🎉 Listo. Ahora estamos en el repo correcto sin duplicados.
 CHECKLIST_INSTALACION.md   notebooks		   run_pipeline.py
 data			   'Pauta proyecto.pdf'    SOLUCION_INSTALACION.md
 DIAGRAMAS_HITO_1.md	    PROBLEMAS_COMUNES.md   src
 examples.py		    QUICK_START.md	   streamlit_app.py
 FIX_CHROMADB.md	    README.md		   test_improvements.py
 install_dependencies.sh    requirements.txt	   TESTING_LOCAL.md
 install_fix.sh		    RESUMEN_FINAL.md	   test_single_pdf.py
 INSTRUCCIONES_PRUEBA.md    RESUMEN_HITO_1.md
 MEJORAS_CHUNKING.md	    RESUMEN_PROBLEMAS.md


# Lesson 2: Document Chunking Strategies

**Objective**: Learn how to effectively segment documents for better retrieval performance.

**Topics**:

- Chunking techniques: token-level, sentence-level, semantic-level
- Balancing context preservation with retrieval precision
- Small2Big and sliding window techniques

**Practical Task**: Implement chunking strategies on a sample dataset.

**Resources**:
- The five levels of chunking
- A guide to chunking
- [Visualizing chunking](https://chunkviz.up.railway.app/)
- [Full stack retrieval](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/a4570f3c4883eb9b835b0ee18990e62298f518ef/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

## Data loading, Chunking techniques and Text processing


### Level 1 - Character splitting

This is the simplest and most basic *chunking* method, it is the process by which we only divide the text into *chunks* of size *N* characters regardless of their content or form.

This method is **NOT** recommended for any application, but it can be a good starting point to understand the fundamentals.

- Pros: easy and fast
- Cons: Very rigid and does not take into account the structure of the text

Concepts:

- **Chunk size**: The number of characters that defines the length of the chunk.
- **Chunk overlap**: The amount of characters that must overlap in sequential chunks. This is to avoid cutting the context into multiple parts, losing the semantic meaning of some passage. This will create duplicates between chunks.

#### Manual example

In [2]:
from pprint import pprint

In [3]:
text = "This is the text I would like to chunk up. It is the example text for this exercise"

# Create an empty list to store the chunks
chunks = []

# Define the chunk size
chunk_size = 35

# Iterate through the text in chunk_size ranges
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
chunks

['This is the text I would like to ch',
 'unk up. It is the example text for ',
 'this exercise']

In [4]:
len(chunks[0])

35

#### Using the LangChain framework

- Normally for RAG applications we don't work with strings, we work with **Document** type objects. These objects contain the text we want but also contain additional metadata that allows for more complex filtering.

In [5]:
from langchain_text_splitters import CharacterTextSplitter

We load the LangChain text splitter, we need to specify the chunk overlap and separator, if we don't provide them we will have unwanted results.

In [6]:
# Sin overlap
text_splitter = CharacterTextSplitter(
                    chunk_size = 35,
                    chunk_overlap=0,
                    separator='',
                    strip_whitespace=True)

In [7]:
text_splitter.create_documents([text])

[Document(metadata={}, page_content='This is the text I would like to ch'),
 Document(metadata={}, page_content='unk up. It is the example text for'),
 Document(metadata={}, page_content='this exercise')]

![image.png](attachment:image.png)

If we observe, we achieved the same result as with the manual option, but now we have data structures that are compatible with the rest of the framework. Additionally, we were able to do some string cleaning, in the case of the whitespace on the second line, using strip_whitespace=True.

In [8]:
# Con overlap
text_splitter = CharacterTextSplitter(
                    chunk_size = 35,
                    chunk_overlap=4,
                    separator='',
                    strip_whitespace=True)
text_splitter.create_documents([text])

[Document(metadata={}, page_content='This is the text I would like to ch'),
 Document(metadata={}, page_content='o chunk up. It is the example text'),
 Document(metadata={}, page_content='ext for this exercise')]

![image.png](attachment:image.png)

If we look closely, we have the same chunks, but between chunk 1 and 2 and chunk 2 and 3 some characters are repeated.

### Level 2: Recursive Character Text Splitting

One of the cons of the previous method is that we don't take into account the structure of the documents. We simply count characters and cut.

**Recursive Character Text Splitter** helps us consider at least the structure of the text, we will specify a series of separators that will be used to separate our document.

These are the available separators:

- "\n\n" - Double line, a common paragraph separator.
- "\n" - Line break, a common sentence separator.
- " " - Whitespace, a common word separator.
- "" - Empty character, a common character separator.

This is the most effective solution to start prototyping an application quickly. If we don't know which one to use, this is the first option to try.

In [9]:
# Imports
from langchain_text_splitters import RecursiveCharacterTextSplitter

Let's test with a longer text

In [10]:
text = """
One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear.

Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor's, you don't get half as many customers. You get no customers, and you go out of business.

It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, the rich get richer. [1]
"""

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 65, chunk_overlap=0)

In [12]:
split_recursive_character = text_splitter.create_documents([text])
split_recursive_character

[Document(metadata={}, page_content="One of the most important things I didn't understand about the"),
 Document(metadata={}, page_content='world when I was a child is the degree to which the returns for'),
 Document(metadata={}, page_content='performance are superlinear.'),
 Document(metadata={}, page_content='Teachers and coaches implicitly told us the returns were linear.'),
 Document(metadata={}, page_content='"You get out," I heard a thousand times, "what you put in." They'),
 Document(metadata={}, page_content='meant well, but this is rarely true. If your product is only'),
 Document(metadata={}, page_content="half as good as your competitor's, you don't get half as many"),
 Document(metadata={}, page_content='customers. You get no customers, and you go out of business.'),
 Document(metadata={}, page_content="It's obviously true that the returns for performance are"),
 Document(metadata={}, page_content='superlinear in business. Some think this is a flaw of'),
 Document(metadata=

If we look closely, several of the chunks end with a **"."**. This is because they are probably the end of a paragraph and the splitter first checks if there is a double line break.

Once the paragraphs are separated, the splitter looks at the chunk size, if it's too big, it separates it by the next separator. If the chunk is still too long, it continues with all the separators.

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 450, chunk_overlap=0)
text_splitter.create_documents([text])

[Document(metadata={}, page_content="One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear."),
 Document(metadata={}, page_content='Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor\'s, you don\'t get half as many customers. You get no customers, and you go out of business.'),
 Document(metadata={}, page_content="It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, th

![image.png](attachment:image.png)



## Level 3: Document Specific Splitting

We continue increasing the complexity, now let's look at different types of documents other than .txt. What happens if we have images? Is it a PDF? Or code?

The previous approaches won't work very well with this type of documents.

This level is all about adjusting the **chunking** strategy to our use case. For now we will only see how we handle complex PDFs. But we can also process other types of files, such as Markdown, Python, JS, etc.

### Markdown

In [17]:
!pip install -U langchain langchain-community langchain-text-splitters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found e

In [1]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [3]:
!pip install "unstructured[md]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=092eb13e759dfdbe4b957c6b68d0261104d3d0647bc6d0af73609822879aa4b7
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [5]:
import os

# Crear carpeta data si no existe
os.makedirs("data", exist_ok=True)

# Crear un archivo markdown de ejemplo
contenido = """# Título de ejemplo

Este es un pequeño documento en **Markdown**.

- Punto 1
- Punto 2
- Punto 3

## Subtítulo

Más texto de ejemplo para probar el chunking.
"""

with open("data/example.md", "w", encoding="utf-8") as f:
    f.write(contenido)

print("Archivo data/example.md creado ✅")


Archivo data/example.md creado ✅


In [6]:
markdown_path = "data/example.md"
loader = UnstructuredMarkdownLoader(markdown_path, mode="elements")
data = loader.load()
data

[Document(metadata={'source': 'data/example.md', 'category_depth': 0, 'languages': ['spa'], 'file_directory': 'data', 'filename': 'example.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-24T02:57:37', 'category': 'Title', 'element_id': '66b7f8c1c2934ce7e1bd492787be2650'}, page_content='Título de ejemplo'),
 Document(metadata={'source': 'data/example.md', 'emphasized_text_contents': ['Markdown'], 'emphasized_text_tags': ['b'], 'languages': ['spa'], 'file_directory': 'data', 'filename': 'example.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-24T02:57:37', 'parent_id': '66b7f8c1c2934ce7e1bd492787be2650', 'category': 'UncategorizedText', 'element_id': '7c9bcd284768d4b68030a3c03fb7b230'}, page_content='Este es un pequeño documento en Markdown.'),
 Document(metadata={'source': 'data/example.md', 'category_depth': 1, 'languages': ['spa'], 'file_directory': 'data', 'filename': 'example.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-24T02:57:37', 'parent_id

In [7]:
pprint(data[6].page_content)

Pretty printing has been turned OFF


In [8]:
pprint(data[6].metadata)


Pretty printing has been turned ON


### Python

In [9]:
from langchain_text_splitters import PythonCodeTextSplitter

In [10]:
python_text = """
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

for i in range(10):
    print (i)
"""

In [11]:
python_splitter = PythonCodeTextSplitter(chunk_size=100, chunk_overlap=0)

In [12]:
pprint(python_splitter.create_documents([python_text]))


Pretty printing has been turned OFF


## Level 4: Semantic Chunking

Considering that the previous methodologies only consider dividing the text into fixed lengths, or defined by static delimiters. Doesn't it seem strange that none of those approaches take into account the content of the documents?

There is a better way.

Embeddings represent the semantic meaning of a string. By themselves, they are not very useful, but when compared with others, it is possible to infer the relationship between different strings.

The hypothesis then is that semantically similar chunks should be grouped together.

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob

# Buscar todos los PDFs de preservantes
file_paths = glob.glob("data/pdfs/*.pdf")

# Cargar documentos
docs = []
for path in file_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

# Chunking semántico básico (por tamaño + overlap)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

len(chunks), chunks[:2]  # mostramos solo los 2 primeros


(0, [])

Para explotar la información contenida en los documentos de preservantes, realizamos un proceso de chunking semántico, que divide el PDF en fragmentos de 1000 caracteres con solape de 200. Esto permite preservar el contexto entre secciones relacionadas, mejorando el rendimiento posterior de los embeddings y del RAG.

In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob

# Buscar todos los PDFs de preservantes
file_paths = glob.glob("data/pdfs/*.pdf")
print("PDFs encontrados:", file_paths)

docs = []
for path in file_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

len(docs), docs[0]


PDFs encontrados: ['data/pdfs/antibiotics-08-00208.pdf', 'data/pdfs/IJFS2018-8410747.pdf', 'data/pdfs/AntimicrobialActivityofSpiceextracts.pdf', 'data/pdfs/FTB-61-212.pdf', 'data/pdfs/fmicb-13-930400.pdf', 'data/pdfs/j.ijfoodmicro.2013.06.025.pdf', 'data/pdfs/The Scientific World Journal - 2022 - Teshome - Potentials of Natural Preservatives to Enhance Food Safety and Shelf Life .pdf', 'data/pdfs/Prop Ca y s. potasio.pdf', 'data/pdfs/Preservantes.pdf', 'data/pdfs/Food aditives1.pdf', 'data/pdfs/processes-11-00590.pdf', 'data/pdfs/j.ijfoodmicro.2020.108952.pdf', 'data/pdfs/1-s2.0-S2405844023042287-main.pdf', 'data/pdfs/Molecules to food, preservatives.pdf', 'data/pdfs/s13568-018-0536-0.pdf', 'data/pdfs/benzoate-sorbate.pdf', 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'data/pdfs/Effects of Acidification and Preservatives on Microbial Growth Puree.pdf', 'data/pdfs/1-s2.0-S2212429225020413-main.pdf']


(475, Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-10-31T16:31:01+08:00', 'author': 'Emiliano J. Quinto, Irma Caro, Luz H. Villalobos-Delgado, Javier Mateo, Beatriz De-Mateo-Silleras and María P. Redondo-Del-Río', 'title': 'Food Safety through Natural Antimicrobials', 'subject': 'Microbial pathogens are the cause of many foodborne diseases after the ingestion of contaminated food. Several preservation methods have been developed to assure microbial food safety, as well as nutritional values and sensory characteristics of food. However, the demand for natural antimicrobial agents is increasing due to consumers’ concern on health issues. Moreover, the use of antibiotics is leading to multidrug resistant microorganisms reinforcing the focus of researchers and the food industry on natural antimicrobials. Natural antimicrobial compounds from plants, animals, bacteria, viruses, algae and mushrooms are covered. Finally, new p

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

basic_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

basic_chunks = basic_splitter.split_documents(docs)

print("Chunks básicos:", len(basic_chunks))
basic_chunks[0]


Chunks básicos: 2145


Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-10-31T16:31:01+08:00', 'author': 'Emiliano J. Quinto, Irma Caro, Luz H. Villalobos-Delgado, Javier Mateo, Beatriz De-Mateo-Silleras and María P. Redondo-Del-Río', 'title': 'Food Safety through Natural Antimicrobials', 'subject': 'Microbial pathogens are the cause of many foodborne diseases after the ingestion of contaminated food. Several preservation methods have been developed to assure microbial food safety, as well as nutritional values and sensory characteristics of food. However, the demand for natural antimicrobial agents is increasing due to consumers’ concern on health issues. Moreover, the use of antibiotics is leading to multidrug resistant microorganisms reinforcing the focus of researchers and the food industry on natural antimicrobials. Natural antimicrobial compounds from plants, animals, bacteria, viruses, algae and mushrooms are covered. Finally, new perspec

In [25]:
##Chunking “estructural” usando títulos / secciones

import re
from langchain_core.documents import Document

# Patrón simple para detectar líneas tipo título (ajústalo según tus PDFs)
pattern_titulo = r"\n([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ0-9 \-]{5,})\n"

def split_by_titles(docs, regex_pattern):
    chunks = []
    for doc in docs:
        text = doc.page_content

        # Separamos por títulos con re.split (manteniendo los títulos)
        parts = re.split(regex_pattern, text)

        # Reconstruimos: título + cuerpo
        # parts = ["textoInicial", "TITULO", "texto1", "TITULO", "texto2", ...]
        for i in range(1, len(parts), 2):
            title = parts[i].strip()
            body = parts[i+1].strip() if i+1 < len(parts) else ""
            if body:
                chunks.append(Document(
                    page_content=f"{title}\n{body}",
                    metadata=doc.metadata
                ))
    return chunks

# Ejecutamos nuestro split por títulos
struct_chunks = split_by_titles(docs, pattern_titulo)

print("Chunks estructurales (por títulos):", len(struct_chunks))
struct_chunks[0]


Chunks estructurales (por títulos): 132


Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240205130724', 'rgid': 'PB:377967974_AS:11431281222134788@1707109932653', 'source': 'data/pdfs/AntimicrobialActivityofSpiceextracts.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='CITATIONS\n3\nREADS\n370\n4 authors:\nHassana Ibrahim Adamu\n6 PUBLICATIONS\xa0\xa0\xa016 CITATIONS')

In [27]:
!pip install -U langchain-experimental sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 5.2 MB/s eta 0:00:00


In [28]:
##Definir embeddings y SemanticChunker

from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# Modelo de embeddings gratuito y liviano
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

semantic_splitter = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile",  # dónde "cortar" según cambios semánticos
    breakpoint_threshold_amount=95,          # 95 = pocos cortes, chunks más largos
)

semantic_chunks = semantic_splitter.split_documents(docs)

print("Chunks semánticos:", len(semantic_chunks))
semantic_chunks[0]


/tmp/ipython-input-4182094111.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chunks semánticos: 1650


Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-10-31T16:31:01+08:00', 'author': 'Emiliano J. Quinto, Irma Caro, Luz H. Villalobos-Delgado, Javier Mateo, Beatriz De-Mateo-Silleras and María P. Redondo-Del-Río', 'title': 'Food Safety through Natural Antimicrobials', 'subject': 'Microbial pathogens are the cause of many foodborne diseases after the ingestion of contaminated food. Several preservation methods have been developed to assure microbial food safety, as well as nutritional values and sensory characteristics of food. However, the demand for natural antimicrobial agents is increasing due to consumers’ concern on health issues. Moreover, the use of antibiotics is leading to multidrug resistant microorganisms reinforcing the focus of researchers and the food industry on natural antimicrobials. Natural antimicrobial compounds from plants, animals, bacteria, viruses, algae and mushrooms are covered. Finally, new perspec

In [29]:
for pct in [80, 90, 95]:
    splitter = SemanticChunker(
        embeddings=embedding_model,
        breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=pct,
    )
    ch = splitter.split_documents(docs)
    print(f"Percentile {pct}: {len(ch)} chunks")


Percentile 80: 4421 chunks
Percentile 90: 2551 chunks
Percentile 95: 1650 chunks


In [ ]:
##Chunking jerárquico (estructura + tamaño)

# 1. Primero, chunking estructural (ya definido antes)
struct_chunks = regex_splitter.split_documents(docs)

print("Secciones detectadas:", len(struct_chunks))

# 2. Luego, sobre esas secciones, un splitter por tamaño
fine_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
)

hier_chunks = fine_splitter.split_documents(struct_chunks)

print("Chunks jerárquicos (estructura + tamaño):", len(hier_chunks))
hier_chunks[0]


In [ ]:
##Comparaciòn de  las tres estrategias
print("Chunks básicos (solo tamaño):    ", len(basic_chunks))
print("Chunks estructurales (títulos):  ", len(struct_chunks))
print("Chunks semánticos (embeddings):  ", len(semantic_chunks))
print("Chunks jerárquicos (2 pasos):    ", len(hier_chunks))


## Level 5: Agentic Chunking

Taking level 4 a bit further, can we give the same instruction to an LLM to do it as a human would?

But, how would a human do chunking? Based on level 4 we could follow these steps:

1. We take a sheet of paper
2. We would start with the first sentence and assume it will be the first chunk, since we don't have any other.
3. We continue reading the document and evaluating if it's a new chunk or part of the previous chunk or an existing one. If it's similar to an existing one, we concatenate it to that chunk, if not, we create a new one.
4. We continue doing this until we finish the document.

In [ ]:
text_document = documents[0].text

In [ ]:
class AgenticChunker:
    def __init__(self, llm):
        self.llm = llm

    def chunk_document(self, text: str, target_chars=2000, overlap_chars=200):
        """Split text into overlapping chunks with summaries."""
        segments = text.split("\n\n")  # Split on paragraph breaks
        full_text = " ".join(segments)
        chunks, start = [], 0

        while start < len(full_text):
            end = start + target_chars
            chunk = full_text[start:end]
            chunks.append(chunk)
            start = max(0, start + target_chars - overlap_chars)

        return [{"text": chunk, "summary": self.generate_summary(chunk)} for chunk in chunks]

    def generate_summary(self, text):
        """Get a concise summary from an LLM."""
        prompt = f"Summarize this text in one sentence:\n\n{text}"
        return self.llm.invoke(prompt)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4.1-mini-2025-04-14', temperature=0.2)

agentic_chunker = AgenticChunker(llm)

In [ ]:
agentics_chunks = agentic_chunker.chunk_document(text_document)

2025-10-24 19:43:14,668 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:16,394 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:17,624 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:18,647 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:19,829 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:20,651 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:21,803 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:22,863 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 19:43:23,825 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [ ]:
pprint(agentics_chunks[0])

{'summary': AIMessage(content='The essay argues that starting a startup is becoming a viable third option for graduating seniors alongside getting a job or attending grad school, with the possibility that undergraduates might increasingly found startups even before graduation, challenging traditional career paths and societal expectations.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 433, 'total_tokens': 480, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUKn3xe8NVr6YVrYQIjaGlVF5oMxV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--39ace1fa-7fc9-48a9-bbf6-ba580183b8db-0', usage_metadata={'input_tokens'

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

2025-10-24 19:43:36,449 - INFO - Use pytorch device_name: mps
2025-10-24 19:43:36,449 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [ ]:
for chunk in agentics_chunks:
    chunk['embedding'] = embeddings.embed_query(chunk['text'])

In [ ]:
pprint(agentics_chunks[0])

{'embedding': [-0.04604353383183479,
               -0.08845345675945282,
               0.06487561017274857,
               -0.033981285989284515,
               0.06916138529777527,
               0.02288164757192135,
               0.0092290248721838,
               0.044321537017822266,
               0.006403755396604538,
               0.02369954250752926,
               0.024176552891731262,
               0.07340338826179504,
               -0.015368692576885223,
               -0.002451920649036765,
               0.03889230266213417,
               -0.03537219762802124,
               -0.022749781608581543,
               -0.14905335009098053,
               0.01162621658295393,
               -0.019916526973247528,
               -0.04435396194458008,
               -0.07741729170084,
               -0.03507914021611214,
               -0.02021920680999756,
               0.07026306539773941,
               0.042740728706121445,
               0.0034428532235324383,
        

# Text processing



### Tokenization, POS and Dependency Parsing

In [ ]:
$ pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0.tar.gz

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "data/Regulaciones cacao y chocolate 2003.pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
pprint(splits[0].page_content)

('Status:  This is the original version (as it was originally made).\n'
 'STATUTORY INSTRUMENTS\n'
 '2003 No. 1659\n'
 'FOOD, ENGLAND\n'
 'The Cocoa and Chocolate Products (England) Regulations 2003\n'
 'Made       -      -      -      - 25th June 2003\n'
 'Laid before Parliament 3rd July 2003\n'
 'Coming into force       -      - 3rd August 2003\n'
 'The Secretary of State, in exercise of the powers conferred by sections '
 '16(1)(e), 17(1), 26(1) and (3)\n'
 'and 48(1) of the Food Safety Act 1990(1) and now vested in him(2) and of all '
 'other powers enabling\n'
 'him in that behalf, having had regard in accordance with section 48(4A) of '
 'that Act to relevant\n'
 'advice given by the Food Standards Agency, and after consultation both as '
 'required by Article 9\n'
 'of Regulation (EC) No. 178/2002 of the European Parliament and of the '
 'Council laying down the\n'
 'general principles and requirements of food law, establishing the European '
 'Food Safety Authority\n'
 'and lay

In [ ]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()
single_doc = nlp(splits[0].page_content)

In [ ]:
def tokenize_text(doc):
    return[
        {
            "text": token.text,
            "position": token.pos_, #part of speech
            "dep": token.dep_, #dependency relation
        }
        for token in doc
    ]

## Token Analysis Function

The `tokenize_text` function extracts linguistic information from a spaCy-processed document:

**Purpose**: Convert spaCy tokens into structured data for text analysis and processing.

**Input**: A spaCy `Doc` object (processed text document)

**Output**: List of dictionaries, where each dictionary contains:

- **`"text"`**: The actual word or punctuation mark (e.g., "The", "cat", ".")
- **`"position"`**: Part-of-speech tag indicating grammatical role:
  - `"DET"` = Determiner (the, a, an)
  - `"NOUN"` = Noun (cat, house, idea)
  - `"VERB"` = Verb (run, think, is)
  - `"ADJ"` = Adjective (quick, beautiful)
  - `"PUNCT"` = Punctuation (., !, ?)
- **`"dep"`**: Dependency relationship showing how words connect:
  - `"det"` = Determiner relationship
  - `"nsubj"` = Nominal subject
  - `"amod"` = Adjectival modifier
  - `"punct"` = Punctuation

**Example Output**:
```python
[
    {'text': 'The', 'position': 'DET', 'dep': 'det'},
    {'text': 'quick', 'position': 'ADJ', 'dep': 'amod'},
    {'text': 'fox', 'position': 'NOUN', 'dep': 'nsubj'},
    {'text': '.', 'position': 'PUNCT', 'dep': 'punct'}
]
```

This structured representation enables intelligent text processing for chunking strategies that preserve linguistic meaning.

In [ ]:
for token in tokenize_text(single_doc):
    print(token)

{'text': 'Status', 'position': 'NOUN', 'dep': 'dep'}
{'text': ':', 'position': 'PUNCT', 'dep': 'punct'}
{'text': ' ', 'position': 'SPACE', 'dep': 'dep'}
{'text': 'This', 'position': 'PRON', 'dep': 'nsubj'}
{'text': 'is', 'position': 'AUX', 'dep': 'ROOT'}
{'text': 'the', 'position': 'DET', 'dep': 'det'}
{'text': 'original', 'position': 'ADJ', 'dep': 'amod'}
{'text': 'version', 'position': 'NOUN', 'dep': 'attr'}
{'text': '(', 'position': 'PUNCT', 'dep': 'punct'}
{'text': 'as', 'position': 'SCONJ', 'dep': 'mark'}
{'text': 'it', 'position': 'PRON', 'dep': 'nsubjpass'}
{'text': 'was', 'position': 'AUX', 'dep': 'auxpass'}
{'text': 'originally', 'position': 'ADV', 'dep': 'advmod'}
{'text': 'made', 'position': 'VERB', 'dep': 'advcl'}
{'text': ')', 'position': 'PUNCT', 'dep': 'punct'}
{'text': '.', 'position': 'PUNCT', 'dep': 'punct'}
{'text': '\n', 'position': 'SPACE', 'dep': 'dep'}
{'text': 'STATUTORY', 'position': 'PROPN', 'dep': 'compound'}
{'text': 'INSTRUMENTS', 'position': 'NOUN', 'dep':

### Stopwords removal


In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

def remove_stopwords(doc):
    return [token for token in doc if token.text.lower() not in spacy_stopwords]

filtered_tokens = remove_stopwords(single_doc)

In [ ]:
# Print the first few filtered tokens to verify
for token in filtered_tokens[:10]:
    print(token.text)

Status
:
 
original
version
(
originally
)
.




### Stemming

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

#example
text = "running runners run easily jumper jumps jumping"

doc = nlp(text)

# apply stemming
stemmed_tokens = [stemmer.stem(token.text) for token in doc]

stem_text = " ".join(stemmed_tokens)

print("original text", text)
print("stemmed text", stem_text)

original text running runners run easily jumper jumps jumping
stemmed text run runner run easili jumper jump jump


### Lemmatization

In [ ]:
def lemmatize_text(doc):
    return [token.lemma_ for token in doc]

lemmatized_tokens = lemmatize_text(single_doc)
print(lemmatized_tokens[:10])

['status', ':', ' ', 'this', 'be', 'the', 'original', 'version', '(', 'as']
